# Using grlc from python
Being written in python itself, it is easy to use grlc from python. Here we show how to use grlc to run a SPARQL query which is stored on github.

First we start by importing grlc (and a couple other libraries we use for working with the data).

In [ ]:
import json
import pandas as pd
from io import StringIO

import grlc
import grlc.utils as utils
import grlc.swagger as swagger

We can load the grlc specification for a github repository. For example, the grlc test query repository https://github.com/CLARIAH/grlc-queries.

In [2]:
user = 'CLARIAH'
repo = 'grlc-queries'
spec, warning = swagger.build_spec(user, repo, git_type=grlc.static.TYPE_GITHUB)

We can have a look at one item of the returned specification.

In [3]:
print(json.dumps(spec[0], indent=2))

{
  "call_name": "/defaults",
  "method": "get",
  "tags": [],
  "summary": "",
  "description": "",
  "params": [
    {
      "name": "endpoint",
      "type": "string",
      "in": "query",
      "description": "Alternative endpoint for SPARQL query",
      "default": "https://dbpedia.org/sparql"
    },
    {
      "name": "genre",
      "type": "string",
      "required": true,
      "in": "query",
      "description": "A value of type string (iri) that will substitute ?_genre_iri in the original query",
      "format": "iri",
      "default": "http://dbpedia.org/resource/Rock_music"
    }
  ],
  "item_properties": null,
  "query": "\nPREFIX dbo: <http://dbpedia.org/ontology/>\nPREFIX dbp: <http://dbpedia.org/property/>\nPREFIX schema: <http://schema.org/>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\n\nSELECT ?band ?album WHERE {\n  ?band rdf:type dbo:Band .\n  ?album rdf:type schema:MusicAlbum .\n  ?band dbo:genre ?_genre_iri .\n  ?album dbp:artist ?band .\n} LIMIT 1

This specification item will execute the following SPARQL query:

In [4]:
print(spec[0]['query'])


PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?band ?album WHERE {
  ?band rdf:type dbo:Band .
  ?album rdf:type schema:MusicAlbum .
  ?band dbo:genre ?_genre_iri .
  ?album dbp:artist ?band .
} LIMIT 100



We can execute this query by calling the API point in our generated API:

In [5]:
print(spec[0]['call_name'])

/defaults


We can use `dispatch_query` functions to load data from a specific query (`description` in this case). For this example, we are loading data in `text/csv` format.

**NOTE:** `description` query loads data from dbpedia.org -- the endpoint is specified in the repository `endpoint.txt` file.

In [6]:
query_name = 'description'
acceptHeader = 'text/csv'

data, code, headers = utils.dispatch_query(user, repo, query_name, git_type=grlc.static.TYPE_GITHUB, acceptHeader=acceptHeader)

Now we just transform these results to a pandas dataframe.

In [7]:
data_grlc = pd.read_csv(StringIO(data))
data_grlc.head(10)

,band,album,genre
0,http://dbpedia.org/resource/Cactus_(American_b...,http://dbpedia.org/resource/Cactus_(Cactus_album),http://dbpedia.org/resource/Blues_rock
1,http://dbpedia.org/resource/Cactus_(American_b...,http://dbpedia.org/resource/Cactus_(Cactus_album),http://dbpedia.org/resource/Hard_rock
2,http://dbpedia.org/resource/Cactus_(American_b...,http://dbpedia.org/resource/Cactus_(Cactus_album),http://dbpedia.org/resource/Heavy_metal_music
3,http://dbpedia.org/resource/Cactus_(American_b...,http://dbpedia.org/resource/Cactus_V,http://dbpedia.org/resource/Blues_rock
4,http://dbpedia.org/resource/Cactus_(American_b...,http://dbpedia.org/resource/Cactus_V,http://dbpedia.org/resource/Hard_rock
5,http://dbpedia.org/resource/Cactus_(American_b...,http://dbpedia.org/resource/Cactus_V,http://dbpedia.org/resource/Heavy_metal_music
6,http://dbpedia.org/resource/Cadet_(band),http://dbpedia.org/resource/Cadet_(album),http://dbpedia.org/resource/Christian_alternat...
7,http://dbpedia.org/resource/Cadet_(band),http://dbpedia.org/resource/Cadet_(album),http://dbpedia.org/resource/Garage_rock
8,http://dbpedia.org/resource/Cadet_(band),http://dbpedia.org/resource/Cadet_(album),http://dbpedia.org/resource/Rockabilly
9,http://dbpedia.org/resource/Café_Tacuba,http://dbpedia.org/resource/Café_Tacuba_(album),http://dbpedia.org/resource/Latin_rock


# Grlc via http

Another alternative is to load data via a running grlc server (in this case `grlc.io`).

In [8]:
import requests

In [9]:
headers = {'accept': 'text/csv'}
resp = requests.get("http://grlc.io/api-git/CLARIAH/grlc-queries/description", headers=headers)

In [10]:
data_requests = pd.read_csv(StringIO(resp.text))
data_requests.head(10)

,band,album,genre
0,http://dbpedia.org/resource/Cactus_(American_b...,http://dbpedia.org/resource/Cactus_(Cactus_album),http://dbpedia.org/resource/Blues_rock
1,http://dbpedia.org/resource/Cactus_(American_b...,http://dbpedia.org/resource/Cactus_(Cactus_album),http://dbpedia.org/resource/Hard_rock
2,http://dbpedia.org/resource/Cactus_(American_b...,http://dbpedia.org/resource/Cactus_(Cactus_album),http://dbpedia.org/resource/Heavy_metal_music
3,http://dbpedia.org/resource/Cactus_(American_b...,http://dbpedia.org/resource/Cactus_V,http://dbpedia.org/resource/Blues_rock
4,http://dbpedia.org/resource/Cactus_(American_b...,http://dbpedia.org/resource/Cactus_V,http://dbpedia.org/resource/Hard_rock
5,http://dbpedia.org/resource/Cactus_(American_b...,http://dbpedia.org/resource/Cactus_V,http://dbpedia.org/resource/Heavy_metal_music
6,http://dbpedia.org/resource/Cadet_(band),http://dbpedia.org/resource/Cadet_(album),http://dbpedia.org/resource/Christian_alternat...
7,http://dbpedia.org/resource/Cadet_(band),http://dbpedia.org/resource/Cadet_(album),http://dbpedia.org/resource/Garage_rock
8,http://dbpedia.org/resource/Cadet_(band),http://dbpedia.org/resource/Cadet_(album),http://dbpedia.org/resource/Rockabilly
9,http://dbpedia.org/resource/Café_Tacuba,http://dbpedia.org/resource/Café_Tacuba_(album),http://dbpedia.org/resource/Latin_rock
